In [ ]:
from datasetsforecast.long_horizon2 import LongHorizon2
import pandas as pd
pd.options.plotting.backend = "plotly"
import warnings
warnings.filterwarnings('ignore')

from datasets_metadata import ts_metadata

In [ ]:
dataset_name = "ETTm2"
Y_df = LongHorizon2.load(directory='data', group=dataset_name)
target_ts = ts_metadata[dataset_name]["target_ts"]
exog_list = ts_metadata[dataset_name]["exog_list"]

In [ ]:
Y_df["ds"] = pd.to_datetime(Y_df["ds"])
Y_df = Y_df.set_index("ds")

# Pivotting original dataset in order to consider other series as exogenous
df_raw = Y_df[Y_df["unique_id"] == target_ts]
for item in Y_df["unique_id"].unique():
    if item == target_ts:
        continue
    df_raw[item] = Y_df[Y_df["unique_id"] == item]["y"]
df_raw = df_raw.reset_index()
df_raw = df_raw.drop("index", axis=1)
df_raw["unique_id"] = f"{target_ts}_raw"

df_raw

In [ ]:
df_raw.drop("unique_id", axis=1).set_index("ds").plot()

In [ ]:
# FARM SHAPING

from pyfarm import farm

target_ts = "y"
df_shaped = df_raw.copy()

for feature in exog_list:
    qts_shaped = farm(refTS=df_raw[target_ts].values, qryTS=df_raw[feature].values, ff_align=False)["qts_shaped"]
    df_shaped[feature] = qts_shaped

df_shaped["unique_id"] = f"{target_ts}_exogenous_FARM_shaped"
df_shaped

In [ ]:
df_shaped.drop("unique_id", axis=1).set_index("ds").plot()

In [ ]:
df = pd.concat([df_raw, df_shaped])
df

In [ ]:
df.to_csv(f"./processed_data/{dataset_name}.csv", index=False)